# Delta Lake Operations - Demo

**Cel szkoleniowy:** Opanowanie podstawowych i zaawansowanych operacji Delta Lake, w tym CRUD, MERGE, Time Travel i optymalizacji.

**Zakres tematyczny:**
- Delta Lake core features: ACID, Delta Log, Schema enforcement
- Schema evolution (additive, automatic)
- Time Travel i Copy-on-write
- CRUD operations: CREATE TABLE, INSERT, UPDATE, DELETE
- MERGE INTO - logika zmian na kluczach
- DESCRIBE DETAIL, DESCRIBE HISTORY
- Optymalizacja: OPTIMIZE, ZORDER BY, VACUUM

## Kontekst i wymagania

- **Dzień szkolenia**: Dzień 2 - Lakehouse & Delta Lake
- **Typ notebooka**: Demo
- **Wymagania techniczne**:
  - Databricks Runtime 13.0+ (zalecane: 14.3 LTS)
  - Unity Catalog włączony
  - Uprawnienia: CREATE TABLE, CREATE SCHEMA, SELECT, MODIFY
  - Klaster: Standard z minimum 2 workers

## Wstęp teoretyczny

**Cel sekcji:** Zrozumienie fundamentów Delta Lake jako formatu tabel transakcyjnych dla data lakehouse.

**Podstawowe pojęcia:**
- **Delta Lake**: Format tabel oparty na Parquet z warstwą transakcyjną (ACID)
- **Delta Log**: Dziennik transakcji przechowujący metadane wszystkich operacji
- **ACID**: Atomicity, Consistency, Isolation, Durability - gwarancje transakcyjne
- **Time Travel**: Możliwość odczytania danych z dowolnego punktu w historii
- **Schema Evolution**: Automatyczne lub kontrolowane zmiany w schemacie tabeli

**Dlaczego to ważne?**
Delta Lake rozwiązuje fundamentalne problemy tradycyjnych data lake'ów: brak transakcji, trudności z aktualizacją danych, brak schema enforcement. Pozwala na reliable data pipelines z gwarancjami ACID i możliwością audytu zmian.

## Izolacja per użytkownik

Uruchom skrypt inicjalizacyjny dla per-user izolacji katalogów i schematów:

In [0]:
%run ../00_setup

## Konfiguracja

Import bibliotek i ustawienie zmiennych środowiskowych:

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window
from datetime import datetime, timedelta

# Konfiguracja zmiennych
CATALOG = "your_catalog"
BRONZE_SCHEMA = "your_bronze_schema"
SILVER_SCHEMA = "your_silver_schema"
GOLD_SCHEMA = "your_gold_schema"
DATASET_BASE_PATH = "your_dataset_base_path"
raw_user = "your_user"

# Wyświetl kontekst użytkownika
print("=== Kontekst użytkownika ===")
print(f"Katalog: {CATALOG}")
print(f"Schema Bronze: {BRONZE_SCHEMA}")
print(f"Schema Silver: {SILVER_SCHEMA}")
print(f"Schema Gold: {GOLD_SCHEMA}")
print(f"Użytkownik: {raw_user}")

# Ustaw katalog i schemat jako domyślne
spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"USE SCHEMA {BRONZE_SCHEMA}")

# Ścieżki do danych źródłowych
CUSTOMERS_CSV = f"{DATASET_BASE_PATH}/customers/customers.csv"
ORDERS_JSON = f"{DATASET_BASE_PATH}/orders/orders_batch.json"

# Nazwy tabel Delta, które będziemy tworzyć
CUSTOMERS_DELTA = f"{BRONZE_SCHEMA}.customers_delta"
ORDERS_DELTA = f"{BRONZE_SCHEMA}.orders_delta"

print(f"\n=== Tabele Delta ===")
print(f"Customers: {CUSTOMERS_DELTA}")
print(f"Orders: {ORDERS_DELTA}")

### Import bibliotek

Importujemy podstawowe biblioteki PySpark do pracy z Delta Lake:
- **functions as F**: Funkcje transformacji danych
- **types**: Definicje typów danych (StructType, StringType)  
- **Window**: Funkcje okienne do analiz
- **datetime**: Operacje na datach i czasie

**Kontekst użytkownika i konfiguracja środowiska:**

### Konfiguracja środowiska

Używamy zmiennych z skryptu `00_setup` i ustawiamy kontekst sesji Spark:


In [ ]:
spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"USE SCHEMA {BRONZE_SCHEMA}")

### Definicja ścieżek

Ustawiamy ścieżki do plików źródłowych i nazwy tabel Delta:

In [ ]:
CUSTOMERS_CSV = f"{DATASET_BASE_PATH}/customers/customers.csv"
ORDERS_JSON = f"{DATASET_BASE_PATH}/orders/orders_batch.json"

CUSTOMERS_DELTA = f"{BRONZE_SCHEMA}.customers_delta"
ORDERS_DELTA = f"{BRONZE_SCHEMA}.orders_delta"

### Weryfikacja konfiguracji

Sprawdzamy poprawność ustawionych zmiennych środowiskowych:

**Wyświetlane zmienne:**
- **catalog**: Izolowany katalog per użytkownik
- **bronze_schema**: Warstwa surowych danych  
- **silver_schema**: Warstwa oczyszczonych danych
- **gold_schema**: Warstwa zagregowanych danych
- **user**: Identyfikator użytkownika

In [ ]:
# Wyświetl konfigurację
display(spark.sql(f"SELECT '{CATALOG}' as catalog, '{BRONZE_SCHEMA}' as bronze_schema, '{SILVER_SCHEMA}' as silver_schema, '{GOLD_SCHEMA}' as gold_schema, '{raw_user}' as user"))

---

## Sekcja 1: Delta Lake Core Features

**Wprowadzenie teoretyczne:**

Delta Lake dodaje warstwę transakcyjną na top of Parquet files. Każda operacja zapisu (INSERT, UPDATE, DELETE, MERGE) jest zapisywana w Delta Log jako transakcja. Delta Log to seria JSON files w folderze `_delta_log/` zawierająca metadane wszystkich zmian.

**Kluczowe pojęcia:**
- **Transaction Log**: Pojedynczy wpis w Delta Log reprezentujący jedną transakcję
- **Checkpoint**: Snapshots stanu tabeli co N transakcji dla szybszego odczytu
- **Schema Enforcement**: Automatyczna walidacja typów danych przy zapisie
- **Optimistic Concurrency**: Wiele czytających, jeden piszący w tym samym czasie

**Zastosowanie praktyczne:**
- Reliable ETL pipelines z gwarancjami ACID
- Incremental data processing z możliwością rollback
- Audit trails - pełna historia zmian w danych

### Przykład 1.1: Utworzenie tabeli Delta z CSV

**Cel:** Demonstracja utworzenia pierwszej tabeli Delta Lake z danych CSV

**Podejście:**
1. Wczytaj dane z CSV do DataFrame
2. Zapisz DataFrame jako tabelę Delta w Unity Catalog
3. Sprawdź metadane tabeli Delta

In [0]:
# Przykład 1.1 - Utworzenie tabeli Delta z CSV

# Wczytaj dane klientów z CSV
customers_df = (
    spark.read
    .format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(CUSTOMERS_CSV)
)

print(f"Wczytano {customers_df.count()} rekordów")
customers_df.printSchema()

# Zapisz jako tabelę Delta
(
    customers_df
    .write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable(CUSTOMERS_DELTA)
)

print(f"\n✓ Utworzono tabelę Delta: {CUSTOMERS_DELTA}")

In [ ]:
customers_df.printSchema()

### Wczytane dane z CSV

**Operacja:** Spark Reader wczytuje plik CSV z opcjami:
- `header=true`: Pierwszy wiersz to nazwy kolumn
- `inferSchema=true`: Automatyczne wykrywanie typów danych  

**Schema:** Sprawdzamy strukturę i typy danych automatycznie wykryte przez Spark

In [ ]:
# Zapisz jako tabelę Delta
(
    customers_df
    .write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable(CUSTOMERS_DELTA)
)

### Zapis jako tabela Delta

**Operacja:** DataFrame Writer zapisuje dane jako tabelę Delta w Unity Catalog:

**Kluczowe opcje:**
- `format("delta")`: Format Delta Lake (Parquet + Delta Log)
- `mode("overwrite")`: Nadpisanie istniejącej tabeli  
- `overwriteSchema=true`: Pozwala na zmianę schematu
- `saveAsTable()`: Rejestracja w Unity Catalog dla governance

**Rezultat:** Utworzona tabela Delta z pełnymi funkcjonalnościami ACID

In [ ]:
(
    customers_df
    .write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable(CUSTOMERS_DELTA)
)

**Wyjaśnienie:**

Powyższy kod wykonuje trzy kluczowe operacje:
1. **Wczytanie danych**: Spark DataFrame API wczytuje CSV z automatycznym wykrywaniem schematu
2. **Zapis jako Delta**: `.format("delta")` zapisuje dane w formacie Delta Lake (Parquet + Delta Log)
3. **Rejestracja w Unity Catalog**: `.saveAsTable()` rejestruje tabelę w katalogu UC, co umożliwia governance

Opcja `overwriteSchema=true` pozwala na zmianę schematu przy ponownym zapisie (użyteczne podczas development).

### Przykład 1.2: Inspektowanie Delta Log

**Cel:** Zrozumienie struktury Delta Log i metadanych tabeli

In [0]:
# Przykład 1.2 - DESCRIBE DETAIL

# Wyświetl szczegółowe informacje o tabeli Delta
detail_df = spark.sql(f"DESCRIBE DETAIL {CUSTOMERS_DELTA}")
display(detail_df)

**Kluczowe metadane w DESCRIBE DETAIL:**
- `format`: Format tabeli (delta)
- `location`: Lokalizacja plików w storage
- `numFiles`: Liczba plików Parquet
- `sizeInBytes`: Rozmiar danych

### Przykład 1.3: Historia transakcji (DESCRIBE HISTORY)

**Cel:** Przegląd pełnej historii operacji na tabeli Delta

In [0]:
history_df = spark.sql(f"DESCRIBE HISTORY {CUSTOMERS_DELTA}")

display(
    history_df.select(
        "version",
        "timestamp",
        "operation",
        "operationParameters",
        "userName"
    )
)

---

## Sekcja 2: CRUD Operations

**Wprowadzenie teoretyczne:**

Delta Lake wspiera pełne operacje CRUD (Create, Read, Update, Delete) na tabelach. W przeciwieństwie do tradycyjnych data lake'ów (immutable files), Delta Lake umożliwia modyfikację i usuwanie rekordów przy zachowaniu gwarancji ACID.

**Kluczowe operacje:**
- **INSERT**: Dodawanie nowych rekordów (append mode)
- **UPDATE**: Modyfikacja istniejących rekordów na podstawie warunku
- **DELETE**: Usuwanie rekordów spełniających warunek
- **MERGE**: Upsert - INSERT nowych + UPDATE istniejących w jednej transakcji

### Przykład 2.1: INSERT - Dodawanie nowych rekordów

**Cel:** Demonstracja append mode - dodanie nowych klientów do tabeli

**Podejście:**
1. Utworzenie DataFrame z nowymi danymi
2. Append do istniejącej tabeli Delta
3. Weryfikacja wyników

In [0]:
# Przykład 2.1 - INSERT (append)

# Policz rekordy przed INSERT
count_before = spark.table(CUSTOMERS_DELTA).count()

### Liczba rekordów przed INSERT

Sprawdzamy aktualną liczbę klientów w tabeli Delta przed dodaniem nowych rekordów.
Ta informacja posłuży do weryfikacji skuteczności operacji INSERT.

**Utwórz nowe dane do dodania:**

In [ ]:
new_customers_data = [
    ("CUST_9001", "Jan", "Kowalski", "jan.kowalski@example.com", "Poland", "2025-01-15"),
    ("CUST_9002", "Anna", "Nowak", "anna.nowak@example.com", "Poland", "2025-01-16"),
    ("CUST_9003", "Piotr", "Wiśniewski", "piotr.wisniewski@example.com", "Poland", "2025-01-17")
]

new_customers_df = spark.createDataFrame(
    new_customers_data,
    ["customer_id", "first_name","last_name","email", "country", "registration_date"]
)

new_customers_df = new_customers_df.select("customer_id","first_name","last_name","email", "country", "registration_date")
display(new_customers_df)

**Wykonaj append nowych klientów:**

In [0]:
new_customers_df.write.mode("append").saveAsTable(CUSTOMERS_DELTA)

In [ ]:
count_after = spark.table(CUSTOMERS_DELTA).count()

### Weryfikacja INSERT

**Operacja wykonana:** Append nowych klientów do tabeli Delta

**Sprawdzenie rezultatu:** Porównanie liczby rekordów przed i po operacji INSERT:
- Przed INSERT: wartość z `count_before`
- Po INSERT: wartość z `count_after`  
- Różnica: liczba dodanych rekordów

**Oczekiwany rezultat:** Zwiększenie liczby rekordów o 3 (dodano 3 nowych klientów)

**Weryfikacja - sprawdź nowo dodane rekordy:**

In [ ]:
display(
    spark.table(CUSTOMERS_DELTA)
    .filter(F.col("customer_id").isin(["CUST_9001", "CUST_9002", "CUST_9003"]))
)

**Wyjaśnienie:**

Operacja INSERT (append mode) dodaje nowe rekordy bez modyfikacji istniejących. Delta Lake:
- Tworzy nowe pliki Parquet z nowymi danymi
- Dodaje transakcję do Delta Log
- Zachowuje atomicity - albo wszystkie rekordy są dodane, albo żaden

Mode `append` jest najbezpieczniejszy - nie modyfikuje istniejących danych.

### Przykład 2.2: UPDATE - Modyfikacja rekordów

**Cel:** Aktualizacja email dla określonych klientów

**Sprawdź rekordy przed UPDATE:**

In [0]:
display(
    spark.table(CUSTOMERS_DELTA)
    .filter(F.col("customer_id") == "CUST_9001")
)

**Wykonaj UPDATE używając Delta Table API:**

In [ ]:
# Utwórz referencję do tabeli Delta
deltaTable = DeltaTable.forName(spark, CUSTOMERS_DELTA)

In [ ]:
from delta.tables import DeltaTable

In [ ]:
# Wykonaj UPDATE
deltaTable.update(
    condition = "customer_id = 'CUST_9001'",
    set = { "email": "'jan.kowalski.NEW@example.com'" }
)

### Wykonanie UPDATE

**Delta Table API:** Używamy `DeltaTable.forName()` i `update()`:

**Parametry UPDATE:**
- `condition`: Warunek WHERE - które rekordy aktualizować
- `set`: Mapowanie kolumn do nowych wartości

**Operacja:** Aktualizacja email dla klienta 'CUST_9001'

**Mechanizm:** Copy-on-write - nowe pliki Parquet + wpis w Delta Log

### Weryfikacja UPDATE

**Sprawdzenie rezultatu:** Po wykonaniu UPDATE sprawdzamy czy email został poprawnie zaktualizowany.

**Oczekiwany rezultat:** Rekord z `customer_id = 'CUST_9001'` powinien mieć nowy email: `jan.kowalski.NEW@example.com`

**Wyświetlenie:** Filtrujemy tabelę po customer_id i sprawdzamy zmianę

In [0]:
# Wyświetl rekordy po UPDATE
display(
    spark.table(CUSTOMERS_DELTA)
    .filter(F.col("customer_id") == "CUST_9001")
)

**Wyjaśnienie:**

UPDATE w Delta Lake:
- Nie modyfikuje istniejących plików Parquet (immutable)
- Tworzy nowe pliki z zaktualizowanymi rekordami
- Oznacza stare pliki jako usunięte w Delta Log
- Copy-on-write semantics zapewnia izolację czytających transakcji

Delta Table API (`DeltaTable.forName()`) jest rekomendowanym sposobem wykonywania UPDATE/DELETE/MERGE.

### Przykład 2.3: DELETE - Usuwanie rekordów

**Cel:** Usunięcie określonych klientów z tabeli Delta

**Krok 1:** Liczenie rekordów przed operacją DELETE

In [0]:
count_before = spark.table(CUSTOMERS_DELTA).count()

### Liczba rekordów przed DELETE

Przechowujemy aktualną liczbę klientów do późniejszego porównania po operacji DELETE.

**Wykonaj DELETE:**

In [0]:
# Przykład 2.3 - DELETE

# Utwórz referencję do tabeli Delta
deltaTable = DeltaTable.forName(spark, CUSTOMERS_DELTA)

# Wykonaj DELETE
deltaTable.delete(
    condition = "customer_id IN ('CUST_9001', 'CUST_9002')"
)

print("✓ DELETE wykonany")


In [ ]:
# Wykonaj DELETE
deltaTable.delete(
    condition = "customer_id IN ('CUST_9001', 'CUST_9002')"
)

### Wykonanie DELETE

**Delta Table API:** Używamy `delete()` z warunkiem:

**Parametry DELETE:**
- `condition`: Warunek WHERE - które rekordy usunąć

**Operacja:** Usunięcie klientów z ID 'CUST_9001' i 'CUST_9002'

**Mechanizm:** Oznaczenie plików jako usunięte w Delta Log (nie fizyczne usunięcie)

### Weryfikacja DELETE

**Sprawdzenie rezultatu:** Porównanie liczby rekordów przed i po operacji DELETE

**Oczekiwany rezultat:** 
- Zmniejszenie liczby rekordów o 2 (usunięte CUST_9001 i CUST_9002)
- Brak rekordów z usuniętymi customer_id (powinno być 0)

In [0]:
print("\n✓ DELETE wykonany")

# Policz rekordy po DELETE
count_after = spark.table(CUSTOMERS_DELTA).count()
print(f"Liczba klientów po DELETE: {count_after}")
print(f"Usunięto: {count_before - count_after} rekordów")

deleted_check = (
    spark.table(CUSTOMERS_DELTA)
    .filter(F.col("customer_id").isin(['CUST_9001', 'CUST_9002']))
    .count()
)
print(f"Liczba rekordów z ID 9001, 9002: {deleted_check} (powinno być 0)")

In [ ]:
# Sprawdź czy rekordy zostały usunięte
deleted_check = (
    spark.table(CUSTOMERS_DELTA)
    .filter(F.col("customer_id").isin(['CUST_9001', 'CUST_9002']))
    .count()
)

---

## Sekcja 3: MERGE INTO - Upsert Logic

**Wprowadzenie teoretyczne:**

MERGE INTO to najbardziej potężna operacja Delta Lake, łącząca INSERT i UPDATE w jednej transakcji. Umożliwia implementację upsert logic: "if record exists - update, else - insert". Jest kluczowa dla Slowly Changing Dimensions (SCD) i incremental ETL.

**Kluczowe koncepty:**
- **Source**: DataFrame z nowymi/zmienionymi danymi
- **Target**: Tabela Delta do aktualizacji
- **Merge Key**: Kolumny do identyfikacji dopasowanych rekordów
- **WHEN MATCHED**: Co zrobić z rekordami, które istnieją w obu
- **WHEN NOT MATCHED**: Co zrobić z nowymi rekordami (tylko w source)

### Przykład 3.1: Podstawowy MERGE (Upsert)

**Cel:** Załaduj nowe zamówienia i zaktualizuj istniejące w jednej operacji

**Podejście:**
1. Utworzenie tabeli orders_delta z danych JSON
2. Przygotowanie nowych/zmienionych zamówień
3. MERGE na kluczu order_id

In [0]:
# Przykład 3.1 - MERGE INTO (podstawowy upsert)

# Krok 1: Utworzenie tabeli orders_delta
# Wczytaj dane zamówień z JSON
orders_df = (
    spark.read
    .format("json")
    .option("multiLine", "true")
    .load(ORDERS_JSON)
)

(
    orders_df
    .write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable(ORDERS_DELTA)
)

print(f"✓ Utworzono tabelę: {ORDERS_DELTA}")
print(f"Liczba zamówień (initial): {spark.table(ORDERS_DELTA).count()}")

In [ ]:
# Zapisz jako tabelę orders_delta
(
    orders_df
    .write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable(ORDERS_DELTA)
)

### Utworzenie tabeli orders_delta

**Operacja:** Wczytanie danych zamówień z pliku JSON i utworzenie tabeli Delta

**Opcje JSON:**
- `multiLine=true`: Obsługa JSON rozłożonego na wiele linii

**Rezultat:** Tabela orders_delta gotowa do operacji MERGE

**Przygotowanie danych do MERGE (nowe i zmienione zamówienia):**

In [0]:
# Krok 2: Przygotowanie danych do MERGE
# Symulacja nowych i zmienionych zamówień

updates_data = [
    # Istniejące zamówienie - zmiana total_amount (UPDATE)
    ("ORD00000001", "CUST005909", "PROD000164", "STORE017", "2024-12-31T23:56:00", 
     1, 206.74, 0, 250.00, "Cash"),
    
    # Nowe zamówienie (INSERT)
    ("ORD99999999", "CUST009999", "PROD001234", "STORE050", "2025-01-20T14:30:00", 
     2, 149.99, 10, 269.98, "Credit Card"),
    
    # Kolejne nowe zamówienie (INSERT)
    ("ORD99999998", "CUST009998", "PROD000567", "STORE025", "2025-01-21T10:15:00", 
     3, 66.50, 5, 189.53, "Debit Card")
]

updates_df = spark.createDataFrame(
    updates_data,
    ["order_id", "customer_id", "product_id", "store_id", "order_datetime",
     "quantity", "unit_price", "discount_percent", "total_amount", "payment_method"]
)

display(updates_df)

**Wykonaj MERGE INTO:**

Merge key: `order_id`
- **WHEN MATCHED**: Aktualizuj `total_amount` i `payment_method`
- **WHEN NOT MATCHED**: Wstaw nowe zamówienie

In [0]:
# Krok 3: Wykonanie MERGE

from delta.tables import DeltaTable

count_initial = spark.table(ORDERS_DELTA).count()

deltaTable = DeltaTable.forName(spark, ORDERS_DELTA)

(
    deltaTable.alias("target")
    .merge(
        updates_df.alias("source"),
        "target.order_id = source.order_id"
    )
    .whenMatchedUpdate(set = {
        "total_amount": "source.total_amount",
        "payment_method": "source.payment_method"
    })
    .whenNotMatchedInsert(values = {
        "order_id": "source.order_id",
        "customer_id": "source.customer_id",
        "product_id": "source.product_id",
        "store_id": "source.store_id",
        "order_datetime": "source.order_datetime",
        "quantity": "source.quantity",
        "unit_price": "source.unit_price",
        "discount_percent": "source.discount_percent",
        "total_amount": "source.total_amount",
        "payment_method": "source.payment_method"
    })
    .execute()
)

count_after = spark.table(ORDERS_DELTA).count()
print(f"✓ MERGE wykonany")
print(f"Liczba zamówień (po MERGE): {count_after}")
print(f"Dodano nowych: {count_after - count_initial}")

In [ ]:
deltaTable = DeltaTable.forName(spark, ORDERS_DELTA)

In [ ]:
# Wykonaj MERGE
(
    deltaTable.alias("target")
    .merge(
        updates_df.alias("source"),
        "target.order_id = source.order_id"
    )
    .whenMatchedUpdate(set = {
        "total_amount": "source.total_amount",
        "payment_method": "source.payment_method"
    })
    .whenNotMatchedInsert(values = {
        "order_id": "source.order_id",
        "customer_id": "source.customer_id",
        "product_id": "source.product_id",
        "store_id": "source.store_id",
        "order_datetime": "source.order_datetime",
        "quantity": "source.quantity",
        "unit_price": "source.unit_price",
        "discount_percent": "source.discount_percent",
        "total_amount": "source.total_amount",
        "payment_method": "source.payment_method"
    })
    .execute()
)

In [ ]:
count_after = spark.table(ORDERS_DELTA).count()

### Rezultat MERGE

**Operacja wykonana:** MERGE INTO z upsert logic

**Sprawdzenie wyników:**
- Liczba zamówień przed MERGE: wartość z `count_initial`
- Liczba zamówień po MERGE: wartość z `count_after`  
- Dodano nowych: różnica między `count_after` a `count_initial`

**Oczekiwany rezultat:** Zwiększenie o 2 zamówienia (nowe ORD99999999 i ORD99999998)

### Weryfikacja - zaktualizowane zamówienie

**Sprawdzenie:** Rekord z `order_id = 'ORD00000001'` powinien mieć zaktualizowane:
- `total_amount`: 250.00 (zmienione z oryginalnej wartości)
- `payment_method`: "Cash" (zmienione z oryginalnej wartości)

**WHEN MATCHED:** Operacja UPDATE została wykonana na istniejącym zamówieniu

In [0]:
# Weryfikacja wyników MERGE

# Weryfikacja - zaktualizowane zamówienie
display(
    spark.table(ORDERS_DELTA)
    .filter(F.col("order_id") == "ORD00000001")
)

### Weryfikacja - nowe zamówienia

**Sprawdzenie:** Nowe zamówienia z ID rozpoczynającymi się od 'ORD99999'

**WHEN NOT MATCHED:** Operacja INSERT została wykonana dla nowych zamówień:
- ORD99999999: Credit Card payment, total_amount: 269.98
- ORD99999998: Debit Card payment, total_amount: 189.53

In [ ]:
display(
    spark.table(ORDERS_DELTA)
    .filter(F.col("order_id").like("ORD99999%"))
)

**Wyjaśnienie:**

MERGE INTO w Delta Lake:
1. **Merge Key**: `order_id` identyfikuje dopasowane rekordy (klucz biznesowy)
2. **WHEN MATCHED**: Aktualizuje `total_amount` i `payment_method` dla istniejących zamówień
3. **WHEN NOT MATCHED**: Wstawia nowe zamówienia z wszystkimi kolumnami ze schematu orders
4. **Atomicity**: Cała operacja (UPDATE + INSERT) jest jedną transakcją ACID

To najbardziej efektywny sposób implementacji incremental data loading w Delta Lake.

**Zalety MERGE vs DELETE + INSERT:**
- Jedna transakcja zamiast dwóch (szybsze, bezpieczniejsze)
- Brak data loss risk przy failure
- Lepsza performance dla dużych tabel
- Automatyczne partition pruning

---

## Sekcja 4: Time Travel

**Wprowadzenie teoretyczne:**

Time Travel to unikalna funkcja Delta Lake umożliwiająca odczyt danych z dowolnego punktu w historii tabeli. Każda wersja tabeli jest identyfikowana przez:
- **Version number**: Liczba całkowita (0, 1, 2, ...)
- **Timestamp**: Data i czas transakcji

**Kluczowe zastosowania:**
- Audyt zmian w danych
- Rollback do poprzedniego stanu po błędzie
- Reprodukowalność analiz (odczyt danych "as of" określonej daty)
- Porównanie zmian między wersjami

### Przykład 4.1: Odczyt historii wersji

**Cel:** Przegląd wszystkich wersji tabeli i ich metadanych

In [0]:
history_df = spark.sql(f"DESCRIBE HISTORY {CUSTOMERS_DELTA}")

display(
    history_df.select(
        "version",
        "timestamp",
        "operation",
        "operationParameters",
        "userName",
        "operationMetrics"
    )
    .orderBy("version", ascending=False)
)

In [ ]:
latest_version = history_df.agg({"version": "max"}).collect()[0][0]

### Najnowsza wersja tabeli

**Operacja:** Znalezienie najwyższego numeru wersji w historii tabeli

**Informacja:** Każda operacja (CREATE, INSERT, UPDATE, DELETE, MERGE) tworzy nową wersję w Delta Log.
Numer wersji zaczyna się od 0 i inkrementuje o 1 przy każdej transakcji.

**Wykorzystanie:** Zmienna `latest_version` zawiera aktualną wersję tabeli

### Przykład 4.2: Odczyt danych z określonej wersji

**Cel:** Użycie Time Travel do odczytania danych z poprzedniej wersji tabeli

In [0]:
# Przykład 4.2 - Odczyt danych @ VERSION AS OF

# Sprawdź liczbę rekordów w najnowszej wersji
current_count = spark.table(CUSTOMERS_DELTA).count()

print(f"Liczba klientów (najnowsza wersja): {current_count}")

# Odczytaj dane z wersji 0
version_0_df = spark.read.format("delta").option("versionAsOf", 0).table(CUSTOMERS_DELTA)
version_0_count = version_0_df.count()

print(f"Liczba klientów (wersja 0): {version_0_count}")
print(f"Różnica: {current_count - version_0_count} rekordów")

In [ ]:
# Odczytaj dane z wersji 0 (pierwotny stan)
version_0_df = spark.read.format("delta").option("versionAsOf", 0).table(CUSTOMERS_DELTA)
version_0_count = version_0_df.count()

### Time Travel - odczyt poprzedniej wersji

**VERSION AS OF syntax:** Odczyt danych z określonej wersji tabeli

**Porównanie:**
- **Aktualna wersja:** `spark.table(CUSTOMERS_DELTA).count()`
- **Wersja 0:** `versionAsOf=0` - pierwotny stan po CREATE TABLE

**Zastosowanie:** Analiza zmian między wersjami, rollback do poprzedniego stanu

### Porównanie wersji

**Analiza zmian:** Różnica między liczbą rekordów w różnych wersjach tabeli

**Interpretacja wyników:**
- **Dodatnia różnica:** Rekordy zostały dodane (INSERT, MERGE INSERT)
- **Ujemna różnica:** Rekordy zostały usunięte (DELETE, MERGE DELETE)
- **Zero:** Brak zmian w liczbie rekordów (UPDATE, MERGE UPDATE)

**Zastosowanie biznesowe:** Monitoring wzrostu danych, audyt operacji

**Odczytaj dane z wersji 0:**

Version 0 to pierwsza wersja tabeli (initial load). Porównamy ją z aktualną wersją.

**Wyświetl dane z wersji 0 (pierwsze 5 rekordów):**

In [ ]:
display(version_0_df.limit(5))

**Alternatywnie - odczyt przez SQL:**

In [ ]:
df_sql = spark.sql(f"SELECT * FROM {CUSTOMERS_DELTA} VERSION AS OF 0 LIMIT 5")
display(df_sql)

### Przykład 4.3: Rollback tabeli (RESTORE)

**Cel:** Przywrócenie tabeli do stanu z poprzedniej wersji

**Uwaga:** RESTORE wykonuje nową transakcję, nie usuwa historii!

In [0]:
# Przykład 4.3 - RESTORE TABLE

# Sprawdź aktualną liczbę rekordów
before_restore = spark.table(CUSTOMERS_DELTA).count()
print(f"Liczba klientów przed RESTORE: {before_restore}")

# Przywróć tabelę do wersji 0 (initial load)
spark.sql(f"RESTORE TABLE {CUSTOMERS_DELTA} TO VERSION AS OF 0")

print("\n✓ RESTORE wykonany")

# Sprawdź liczbę rekordów po RESTORE
after_restore = spark.table(CUSTOMERS_DELTA).count()
print(f"Liczba klientów po RESTORE: {after_restore}")

# Sprawdź historię - RESTORE dodaje nową transakcję!
history_after_restore = spark.sql(f"DESCRIBE HISTORY {CUSTOMERS_DELTA}")
print("\n=== Historia po RESTORE ===")
display(
    history_after_restore
    .select("version", "timestamp", "operation", "operationParameters")
    .orderBy("version", ascending=False)
    .limit(3)
)

print("\nUwaga: RESTORE utworzył nową wersję - historia NIE została utracona!")

**Wykonaj RESTORE do wersji 0:**

In [ ]:
spark.sql(f"RESTORE TABLE {CUSTOMERS_DELTA} TO VERSION AS OF 0")
print("✓ RESTORE wykonany")

**Weryfikacja - sprawdź liczbę rekordów po RESTORE:**

In [ ]:
after_restore = spark.table(CUSTOMERS_DELTA).count()
print(f"Liczba klientów po RESTORE: {after_restore}")

**Sprawdź historię - RESTORE dodaje nową transakcję:**

In [ ]:
history_after_restore = spark.sql(f"DESCRIBE HISTORY {CUSTOMERS_DELTA}")
display(
    history_after_restore
    .select("version", "timestamp", "operation", "operationParameters")
    .orderBy("version", ascending=False)
    .limit(3)
)

print("Uwaga: RESTORE utworzył nową wersję - historia NIE została utracona!")

---

## Sekcja 5: Schema Evolution

**Wprowadzenie teoretyczne:**

Schema Evolution to mechanizm automatycznej adaptacji schematu tabeli Delta do zmian w danych. Delta Lake wspiera:
- **Additive schema changes**: Dodawanie nowych kolumn
- **Schema enforcement**: Blokowanie niekompatybilnych zmian typu
- **Schema merge**: Automatyczne dodawanie kolumn przy zapisie

**Kluczowe opcje:**
- `mergeSchema=true`: Automatyczne dodawanie nowych kolumn z source DataFrame
- `overwriteSchema=true`: Całkowite zastąpienie schematu (uwaga: destructive!)

**Kiedy używać:**
- Dodawanie nowych atrybutów do danych bez przebudowy pipeline'u
- Evolving data models w zgodzie z business requirements
- Integracja nowych źródeł danych z dodatkowymi polami

### Przykład 5.1: Automatyczne dodawanie nowych kolumn

**Cel:** Demonstracja additive schema evolution z opcją `mergeSchema`

In [0]:
# Przykład 5.1 - Schema Evolution z mergeSchema

# KROK 1: Sprawdź aktualny schemat tabeli
spark.table(CUSTOMERS_DELTA).printSchema()

current_columns = len(spark.table(CUSTOMERS_DELTA).columns)
print(f"Aktualna liczba kolumn: {current_columns}")

**Utwórz DataFrame z nowymi kolumnami:**

Nowe kolumny: `loyalty_tier`, `preferred_language`

In [0]:
extended_customers = spark.createDataFrame([
    ("CUST_9999", "Ewa", "Wiśniewska", "ewa.wisn@example.com", "+48 600 123 456",
     "Warsaw", "mazowieckie", "Poland", "2025-01-25", "Premium", "Premium", "PL")
], ["customer_id", "first_name", "last_name", "email", "phone", "city", "state", 
    "country", "registration_date", "customer_segment", "loyalty_tier", "preferred_language"])

print("Nowe kolumny: loyalty_tier, preferred_language")
extended_customers.printSchema()



**Próba zapisu bez mergeSchema (spowoduje błąd):**

In [0]:
try:
    (
        extended_customers
        .write
        .format("delta")
        .mode("append")
        .saveAsTable(CUSTOMERS_DELTA)
    )
except Exception as e:
    print(f"❌ Błąd (oczekiwany): {str(e)[:200]}...")
    print("Delta Lake blokuje niezgodne schematy!")

**Zapis z mergeSchema=true (automatyczne dodanie kolumn):**

In [0]:
# Zapis z mergeSchema - automatyczne dodanie kolumn (registration_date (DATE))
(
    extended_customers
    .write
    .format("delta")
    .mode("append")
    .option("mergeSchema", "true")
    .saveAsTable(CUSTOMERS_DELTA)
)

print("✓ Zapis z mergeSchema=true wykonany")

**Schemat po schema evolution:**

In [0]:
spark.table(CUSTOMERS_DELTA).printSchema()

**Nowy rekord z dodatkowymi kolumnami:**

In [0]:
display(
    spark.table(CUSTOMERS_DELTA)
    .filter(F.col("customer_id") == "CUST_9999")
    .select("customer_id", "first_name", "last_name", "email", "customer_segment", 
            "loyalty_tier", "preferred_language")
)

**Stare rekordy (mają NULL w nowych kolumnach):**

In [ ]:
display(
    spark.table(CUSTOMERS_DELTA)
    .filter(F.col("customer_id").like("CUST000%"))
    .select("customer_id", "first_name", "last_name", "customer_segment", 
            "loyalty_tier", "preferred_language")
    .limit(5)
)

---

## Sekcja 6: Optymalizacja tabeli Delta

**Wprowadzenie teoretyczne:**

Delta Lake akumuluje wiele małych plików przy częstych zapisach. Optymalizacja jest kluczowa dla performance:

**OPTIMIZE (Compaction):**
- Łączy małe pliki Parquet w większe (target: 1 GB)
- Zmniejsza overhead odczytu (mniej plików = mniej I/O operations)
- Poprawia query performance

**ZORDER BY:**
- Co-locality algorytm dla multi-dimensional clustering
- Organizuje dane wg często filtrowanych kolumn
- Zmniejsza data skipping overhead

**VACUUM:**
- Usuwa stare pliki Parquet nieużywane przez aktywne wersje
- Zwalnia storage space
- Domyślnie: retention 7 dni (chroni Time Travel)

**Uwaga:** Po VACUUM nie można odczytać starszych wersji poza retention period!

## Optymalizacja tabel Delta – gdzie szukać szczegółów?

Delta Lake oferuje mechanizmy optymalizacji wydajności, m.in.:

- **OPTIMIZE** – łączenie małych plików w większe,
- **ZORDER BY** – lepszy data skipping dla często filtrowanych kolumn,
- **VACUUM** – usuwanie starych plików i utrzymywanie historii.

Szczegółowe przykłady i best practices znajdziesz w notebooku:
**`05_optimization_best_practices`** (Dzień 2).

---

## Sekcja 7: Change Data Feed (CDF)

**Wprowadzenie teoretyczne:**

**Change Data Feed (CDF)** to feature Delta Lake umożliwiający śledzenie wszystkich zmian (inserts, updates, deletes) w tabeli Delta. CDF rejestruje:
- **Typ zmiany**: insert, update_preimage, update_postimage, delete
- **Zmienione dane**: before/after values
- **Metadata**: version, timestamp operacji

**Kluczowe koncepty:**

**1. Change Types:**
```
- insert: Nowy rekord dodany
- update_preimage: Stan rekordu PRZED update
- update_postimage: Stan rekordu PO update
- delete: Rekord usunięty
```

**2. CDF vs Time Travel:**
| Feature | Time Travel | Change Data Feed |
|---------|-------------|------------------|
| **Use Case** | Snapshot całej tabeli | Tylko zmiany (delta) |
| **Performance** | Odczyt całej tabeli | Odczyt tylko zmian |
| **Format** | Full table | Change rows only |
| **Best For** | Rollback, audit | Incremental ETL, CDC |

**3. Kiedy używać CDF:**
- ✅ **Incremental ETL**: Process tylko nowe/zmienione rekordy
- ✅ **CDC (Change Data Capture)**: Replikacja zmian do downstream systems
- ✅ **Audit trail**: Pełna historia zmian dla compliance
- ✅ **Materialized views**: Incrementally refresh aggregates
- ✅ **Event streaming**: Publish changes to Kafka/EventHub

**4. CDF Architecture:**
```
Delta Table → CDF Enabled → Change Log
                                ↓
              Read via table_changes() function
                                ↓
           INSERT/UPDATE/DELETE events
```

**Włączenie CDF:**
```sql
-- Method 1: Przy tworzeniu tabeli
CREATE TABLE table_name (...) 
TBLPROPERTIES (delta.enableChangeDataFeed = true);

-- Method 2: Dla istniejącej tabeli
ALTER TABLE table_name 
SET TBLPROPERTIES (delta.enableChangeDataFeed = true);
```

**Odczyt CDF:**
```python
# PySpark API
changes_df = spark.read.format("delta") \
    .option("readChangeFeed", "true") \
    .option("startingVersion", 0) \
    .table("table_name")

# SQL API
SELECT * FROM table_changes('table_name', 0)  -- Od wersji 0
SELECT * FROM table_changes('table_name', 0, 10)  -- Od wersji 0 do 10
```

**Change Schema:**
```
Standard columns + CDF metadata:
- _change_type: insert | update_preimage | update_postimage | delete
- _commit_version: Version number
- _commit_timestamp: Timestamp operacji
```

**Best Practices:**
1. **Enable CDF early**: Najlepiej przy CREATE TABLE (nie można odzyskać historii sprzed włączenia)
2. **Incremental reads**: Używaj `startingVersion`/`endingVersion` dla efficiency
3. **Storage overhead**: CDF dodaje ~30% storage (change data stored separately)
4. **Retention**: CDF podlega tej samej retention policy co VACUUM

---

## Best Practices

**Organizacja tabel Delta:**
1. **Naming convention**: Używaj `{layer}.{domain}_{entity}` (np. `bronze.sales_orders`)
2. **Partitioning**: Partycjonuj tylko duże tabele (>1TB) po często filtrowanych kolumnach (np. date)
3. **Table properties**: Ustaw `delta.autoOptimize.optimizeWrite = true` dla częstych małych zapisów

**Optymalizacja performance:**
1. **OPTIMIZE**: Uruchamiaj regularnie (np. co noc) dla tabel z częstymi zapisami
2. **ZORDER BY**: Używaj dla 2-4 najczęściej filtrowanych kolumn (więcej = diminishing returns)
3. **Data skipping**: Wykorzystuj statistics w Delta Log - filtruj po kolumnach z ZORDER

**CRUD operations:**
1. **MERGE**: Preferuj zamiast DELETE + INSERT dla upsert logic (atomicity!)
2. **Merge keys**: Zawsze używaj indeksowanych kolumn (np. primary keys)
3. **Predicates**: Dodawaj dodatkowe predykaty w MERGE dla partition pruning

**Time Travel i maintenance:**
1. **VACUUM retention**: Minimum 7 dni (domyślnie) - chroni Time Travel i concurrent readers
2. **DESCRIBE HISTORY**: Monitoruj operacje i rozmiar Delta Log
3. **Checkpoint**: Tworzony automatycznie co 10 transakcji - nie wymaga interwencji

**Schema evolution:**
1. **mergeSchema**: Używaj ostrożnie - może spowodować NULL-e w starych rekordach
2. **overwriteSchema**: Tylko dla development - destrukcyjna operacja!
3. **NOT NULL constraints**: Definiuj przed pierwszym zapisem (trudno dodać później)

---

## Troubleshooting

**Problem 1: "Schema mismatch" przy zapisie**
```
AnalysisException: A schema mismatch detected when writing to the Delta table
```
**Rozwiązanie:**
- Użyj `option("mergeSchema", "true")` dla additive changes
- Użyj `option("overwriteSchema", "true")` dla pełnej zmiany schematu (uwaga: destructive!)

**Problem 2: "ConcurrentAppendException"**
```
ConcurrentAppendException: Files were added to the table by a concurrent update
```
**Rozwiązanie:**
- Delta Lake używa optimistic concurrency - retry operacji
- Dla częstych konfliktów: zastosuj partition pruning lub MERGE z predykatami

**Problem 3: Time Travel nie działa dla starszych wersji**
```
VersionNotFoundException: Cannot find version X
```
**Rozwiązanie:**
- Sprawdź czy VACUUM nie usunął starych plików
- Zwiększ retention period: `VACUUM table RETAIN 30 DAYS`

**Problem 4: Słaba performance query po wielu UPDATE/DELETE**
**Rozwiązanie:**
- Uruchom `OPTIMIZE` dla compaction małych plików
- Użyj `OPTIMIZE ZORDER BY` dla często filtrowanych kolumn

**Problem 5: Duże zużycie storage pomimo VACUUM**
**Rozwiązanie:**
- Sprawdź `DESCRIBE DETAIL` - czy Delta Log nie rośnie?
- Checkpoint tworzony automatycznie co 10 transakcji
- Rozważ manual checkpoint: `OPTIMIZE table` tworzy checkpoint jako side effect

---

## Podsumowanie

**W tym notebooku nauczyliśmy się:**

✅ **Delta Lake Core Features:**
- Transakcyjna warstwa ACID dla data lakehouse
- Delta Log jako dziennik metadanych wszystkich operacji
- Schema enforcement i automatic schema evolution

✅ **CRUD Operations:**
- INSERT (append mode) - dodawanie nowych rekordów
- UPDATE - modyfikacja istniejących danych z Copy-on-Write semantics
- DELETE - usuwanie rekordów spełniających warunek
- MERGE INTO - upsert logic w jednej transakcji ACID

✅ **Time Travel:**
- Odczyt danych z dowolnej wersji: `VERSION AS OF`, `TIMESTAMP AS OF`
- RESTORE TABLE - rollback do poprzedniego stanu (tworzy nową wersję!)
- DESCRIBE HISTORY - audyt wszystkich operacji na tabeli

✅ **Schema Evolution:**
- Additive changes z `mergeSchema=true`
- Automatyczna adaptacja do nowych kolumn w danych źródłowych
- Schema enforcement chroni przed niekompatybilnymi zmianami

✅ **Optymalizacja:**
- OPTIMIZE - compaction małych plików dla performance
- ZORDER BY - multi-dimensional clustering dla data skipping
- VACUUM - usuwanie nieużywanych plików (uwaga: ogranicza Time Travel!)

**Kluczowe wnioski:**
1. Delta Lake rozwiązuje fundamentalne problemy tradycyjnych data lake'ów (brak transakcji, trudności z UPDATE/DELETE)
2. MERGE INTO jest kluczową operacją dla incremental ETL i Slowly Changing Dimensions
3. Time Travel umożliwia audyt, rollback i reprodukowalność analiz
4. Regularna optymalizacja (OPTIMIZE, VACUUM) jest niezbędna dla production workloads
5. Schema evolution pozwala na evolving data models bez przebudowy pipeline'ów

**Następne kroki:**
- Delta Live Tables - deklaratywne pipeline'y z automatycznym dependency management
- Change Data Feed - incremental processing z CDC patterns
- Delta Sharing - bezpieczne udostępnianie danych bez kopiowania

---

## Cleanup

Opcjonalnie: usuń utworzone tabele Demo po zakończeniu ćwiczeń:

**Uwaga:** Wykonaj cleanup tylko jeśli nie potrzebujesz już tych tabel!

In [0]:
# Cleanup - usuń tabele demo

# Odkomentuj poniższe linie aby usunąć tabele:

# spark.sql(f"DROP TABLE IF EXISTS {CUSTOMERS_DELTA}")
# spark.sql(f"DROP TABLE IF EXISTS {ORDERS_DELTA}")
# print("✓ Tabele usunięte")

print("Cleanup wyłączony (odkomentuj kod aby usunąć tabele)")

In [ ]:
print("Cleanup wyłączony (odkomentuj kod aby usunąć tabele)")

**Status cleanup:**

Cleanup jest wyłączony - tabele pozostają zachowane do dalszych eksperymentów. Odkomentuj kod powyżej aby usunąć tabele demo.

### Przykład 7.1: Włączenie Change Data Feed

**Cel:** Włączenie CDF dla tabeli orders_delta i wykonanie operacji CRUD

In [0]:
# Przykład 7.1 - Włączenie Change Data Feed

# Krok 1: Sprawdź czy CDF jest włączony
print("=== Sprawdzenie stanu CDF ===")
table_props = spark.sql(f"SHOW TBLPROPERTIES {ORDERS_DELTA}").collect()
cdf_enabled = any(row['key'] == 'delta.enableChangeDataFeed' and row['value'] == 'true' for row in table_props)

print(f"CDF włączony: {cdf_enabled}")

# Krok 2: Włącz CDF
if not cdf_enabled:
    spark.sql(f"""
        ALTER TABLE {ORDERS_DELTA}
        SET TBLPROPERTIES (delta.enableChangeDataFeed = true)
    """)
    print("✓ CDF włączony dla tabeli orders_delta")
else:
    print("✓ CDF już był włączony")

In [ ]:
if not cdf_enabled:
    spark.sql(f"""
        ALTER TABLE {ORDERS_DELTA}
        SET TBLPROPERTIES (delta.enableChangeDataFeed = true)
    """)
    print("✓ CDF włączony dla tabeli orders_delta")
else:
    print("✓ CDF już był włączony")

**Włącz CDF dla tabeli orders_delta:**

Change Data Feed musi być włączony przed wykonaniem operacji CRUD, które chcemy śledzić.

In [ ]:
print(f"Start version dla CDF tracking: {start_version}")

**Start version obliczony:**

Ta wersja będzie punktem startowym dla śledzenia zmian CDF. Wszystkie kolejne operacje CRUD będą zarejestrowane w Change Data Feed.

In [ ]:
new_orders.write.mode("append").saveAsTable(ORDERS_DELTA)
print("✓ INSERT: 2 nowe zamówienia dodane")

**Wykonaj INSERT (append nowych zamówień):**

Te zamówienia zostaną dodane jako nowe rekordy i zarejestrowane w CDF jako events typu `insert`.

In [ ]:
print("✓ UPDATE: Zmieniono payment_method i total_amount dla ORD_CDF_001")

**UPDATE wykonany:**

CDF zarejestruje dwa events: `update_preimage` (stan przed) i `update_postimage` (stan po zmianie).

In [ ]:
print("✓ DELETE: Usunięto zamówienie ORD_CDF_002")

**DELETE wykonany:**

CDF zarejestruje event typu `delete` z ostatnimi wartościami usuniętego rekordu.

In [ ]:
print(f"Start version: {start_version}")
print(f"End version: {end_version}")
print(f"Liczba transakcji tracked: {end_version - start_version + 1}")
print("\nOperacje wykonane:")
print("  1. INSERT (2 records)")
print("  2. UPDATE (1 record)")
print("  3. DELETE (1 record)")

**Podsumowanie tracked operacji:**

Wszystkie wykonane operacje CRUD zostały zarejestrowane w Change Data Feed między wersjami start a end.

In [ ]:
print(f"CDF włączony: {cdf_enabled}")

**Status CDF dla tabeli:**

CDF jest obecnie wyłączony - musimy go włączyć za pomocą ALTER TABLE.

**Zapisz start version dla CDF tracking:**

In [ ]:
history = spark.sql(f"DESCRIBE HISTORY {ORDERS_DELTA}")
start_version = history.agg({"version": "max"}).collect()[0][0] + 1

print(f"Start version dla CDF tracking: {start_version}")

**Operacja 1: INSERT (nowe zamówienia):**

In [ ]:
new_orders = spark.createDataFrame([
    ("ORD_CDF_001", "CUST001234", "PROD000123", "STORE010", "2025-01-22T10:00:00", 
     5, 99.99, 10, 449.96, "Credit Card"),
    ("ORD_CDF_002", "CUST001235", "PROD000456", "STORE020", "2025-01-22T11:30:00", 
     2, 149.50, 5, 284.05, "Debit Card")
], ["order_id", "customer_id", "product_id", "store_id", "order_datetime",
    "quantity", "unit_price", "discount_percent", "total_amount", "payment_method"])

**Operacja 2: UPDATE (zmiana payment_method):**

In [ ]:
spark.sql(f"""
    UPDATE {ORDERS_DELTA}
    SET payment_method = 'Cash', total_amount = 400.00
    WHERE order_id = 'ORD_CDF_001'
""")

**Operacja 3: DELETE (usunięcie zamówienia):**

In [ ]:
spark.sql(f"""
    DELETE FROM {ORDERS_DELTA}
    WHERE order_id = 'ORD_CDF_002'
""")

**Podsumowanie wykonanych operacji:**

In [ ]:
end_version = spark.sql(f"DESCRIBE HISTORY {ORDERS_DELTA}").agg({"version": "max"}).collect()[0][0]

operations = [
    {"type": "INSERT", "count": 2},
    {"type": "UPDATE", "count": 1},
    {"type": "DELETE", "count": 1},
]

print(f"Start version: {start_version}")
print(f"End version: {end_version}")
print(f"Liczba transakcji tracked: {end_version - start_version + 1}")
print("\nOperacje wykonane:")
for operation in operations:
    print(f"  {operation['type']} ({operation['count']} records)")

### Przykład 7.2: Odczyt Change Data Feed

**Cel:** Odczyt i analiza zmian z CDF - wszystkie typy change events

In [0]:
# Przykład 7.2 - Odczyt Change Data Feed

# Metoda 1: PySpark API (readChangeFeed)
print("=== Metoda 1: PySpark API ===")

# Odczytaj zmiany od start_version używając PySpark API
changes_df = (spark.read
    .format("delta")
    .option("readChangeFeed", "true")
    .option("startingVersion", start_version)
    .table(ORDERS_DELTA)
)

print(f"Liczba change events: {changes_df.count()}")

In [ ]:
print(f"Liczba change events: {changes_df.count()}")

**Liczba zarejestrowanych change events:**

CDF rejestruje każdą operację CRUD jako separate events. UPDATE generuje 2 events (preimage + postimage).

**Schema CDF (zauważ dodatkowe kolumny):**

Kolumny CDF:
- `_change_type`: insert | update_preimage | update_postimage | delete
- `_commit_version`: Version number
- `_commit_timestamp`: Timestamp operacji

In [ ]:
display(
    changes_df
    .select(
        "order_id", "customer_id", "total_amount", "payment_method",
        "_change_type", "_commit_version", "_commit_timestamp"
    )
    .orderBy("_commit_version", "_change_type")
)

**Podsumowanie zmian po typach:**

In [ ]:
change_summary = changes_df.groupBy("_change_type").count().orderBy("_change_type")
display(change_summary)

### Przykład 7.3: Use Case - Incremental ETL z CDF

**Cel:** Demonstracja praktycznego użycia CDF dla incremental processing

**Scenario:** 
- Source table (orders_delta) zmienia się często
- Target table (orders_summary) agreguje zamówienia
- Chcemy aktualizować summary tylko dla zmienionych rekordów (nie full refresh!)

**Podejście:**
1. Read CDF od ostatniej processed version
2. Process tylko changed records
3. Update target table incrementally

In [0]:
# Przykład 7.3 - Incremental ETL z CDF

print("=== Use Case: Incremental ETL z CDF ===\n")

# Krok 1: Przygotuj target table (orders_summary)
ORDERS_SUMMARY = f"{GOLD_SCHEMA}.orders_summary_cdf"

# Initial load - full aggregation (tylko raz)
print("Krok 1: Initial load (full aggregation)")
initial_summary = (spark.table(ORDERS_DELTA)
    .groupBy("customer_id", "payment_method")
    .agg(
        F.count("*").alias("order_count"),
        F.sum("total_amount").alias("total_spent"),
        F.avg("total_amount").alias("avg_order_value")
    )
)

initial_summary.write.format("delta").mode("overwrite").saveAsTable(ORDERS_SUMMARY)
print(f"✓ Utworzono {ORDERS_SUMMARY}")
print(f"  Records: {spark.table(ORDERS_SUMMARY).count()}")

# Zapisz last processed version
last_processed_version = spark.sql(f"DESCRIBE HISTORY {ORDERS_DELTA}") \
    .agg({"version": "max"}).collect()[0][0]
print(f"  Last processed version: {last_processed_version}\n")

# Krok 2: Symulacja nowych zmian w source table
print("Krok 2: Symulacja nowych zmian w orders_delta")

# Dodaj nowe zamówienia
new_orders_2 = spark.createDataFrame([
    ("ORD_CDF_003", "CUST001234", "PROD000789", "STORE030", "2025-01-23T09:00:00", 
     3, 66.66, 0, 199.98, "Credit Card"),
    ("ORD_CDF_004", "CUST001236", "PROD000111", "STORE040", "2025-01-23T10:00:00", 
     1, 299.99, 15, 254.99, "Cash")
], ["order_id", "customer_id", "product_id", "store_id", "order_datetime",
    "quantity", "unit_price", "discount_percent", "total_amount", "payment_method"])

new_orders_2.write.mode("append").saveAsTable(ORDERS_DELTA)
print("✓ Dodano 2 nowe zamówienia\n")

# Krok 3: Incremental processing z CDF
print("Krok 3: Incremental processing (tylko zmiany!)")

current_version = spark.sql(f"DESCRIBE HISTORY {ORDERS_DELTA}") \
    .agg({"version": "max"}).collect()[0][0]

# Read tylko changes od last processed version
changes_incremental = (spark.read
    .format("delta")
    .option("readChangeFeed", "true")
    .option("startingVersion", last_processed_version + 1)
    .table(ORDERS_DELTA)
)

print(f"CDF changes (versions {last_processed_version + 1} to {current_version}):")
print(f"  Change events: {changes_incremental.count()}")

# Filter tylko inserts (dla simplicity - w production obsłużyć update/delete)
inserts_only = changes_incremental.filter(F.col("_change_type") == "insert")

if inserts_only.count() > 0:
    # Compute incremental aggregates
    incremental_agg = (inserts_only
        .groupBy("customer_id", "payment_method")
        .agg(
            F.count("*").alias("order_count"),
            F.sum("total_amount").alias("total_spent"),
            F.avg("total_amount").alias("avg_order_value")
        )
    )
    
    print(f"  Incremental aggregates: {incremental_agg.count()} rows")
    display(incremental_agg)
    
    # MERGE incremental aggregates into summary table
    from delta.tables import DeltaTable
    
    summary_table = DeltaTable.forName(spark, ORDERS_SUMMARY)
    
    (summary_table.alias("target")
        .merge(
            incremental_agg.alias("source"),
            "target.customer_id = source.customer_id AND target.payment_method = source.payment_method"
        )
        .whenMatchedUpdate(set = {
            "order_count": "target.order_count + source.order_count",
            "total_spent": "target.total_spent + source.total_spent",
            "avg_order_value": "(target.total_spent + source.total_spent) / (target.order_count + source.order_count)"
        })
        .whenNotMatchedInsert(values = {
            "customer_id": "source.customer_id",
            "payment_method": "source.payment_method",
            "order_count": "source.order_count",
            "total_spent": "source.total_spent",
            "avg_order_value": "source.avg_order_value"
        })
        .execute()
    )
    
    print("\n✓ Incremental MERGE wykonany!")
else:
    print("  Brak nowych insertów do przetworzenia")

# Krok 4: Weryfikacja
print("\n=== Podsumowanie ===")
print(f"✅ Incremental ETL zakończony pomyślnie!")
print(f"   Przetworzone wersje: {last_processed_version + 1} → {current_version}")
print(f"   Tylko {inserts_only.count()} change events (nie full table scan!)")
print(f"\n💡 Korzyści CDF:")
print(f"   • Performance: Odczyt tylko zmian (nie full table)")
print(f"   • Efficiency: Incremental processing")
print(f"   • Scalability: Działa dla billion-row tables")

print("\n=== Orders Summary (po incremental update) ===")
display(spark.table(ORDERS_SUMMARY).orderBy(F.desc("total_spent")))

In [ ]:
display(spark.table(ORDERS_SUMMARY).orderBy(F.desc("total_spent")))

**Orders Summary (po incremental update):**

Wynik incremental ETL - tabela podsumowań została zaktualizowana tylko na podstawie zmian z CDF.

In [ ]:
print("=== Podsumowanie ===")
print(f"✅ Incremental ETL zakończony pomyślnie!")
print(f"Przetworzone wersje: {last_processed_version + 1} → {current_version}")
print(f"Tylko {inserts_count} change events (nie full table scan!)")
print("\n💡 Korzyści CDF:")
print("• Performance: Odczyt tylko zmian (nie full table)")
print("• Efficiency: Incremental processing")
print("• Scalability: Działa dla billion-row tables")

**Podsumowanie incremental ETL:**

Dzięki CDF przetworzyliśmy tylko zmienione rekordy zamiast całej tabeli - znacznie lepsze performance dla dużych datasets.

In [ ]:
if inserts_count > 0:
    from delta.tables import DeltaTable
    
    summary_table = DeltaTable.forName(spark, ORDERS_SUMMARY)
    
    (summary_table.alias("target")
        .merge(
            incremental_agg.alias("source"),
            "target.customer_id = source.customer_id AND target.payment_method = source.payment_method"
        )
        .whenMatchedUpdate(set = {
            "order_count": "target.order_count + source.order_count",
            "total_spent": "target.total_spent + source.total_spent",
            "avg_order_value": "(target.total_spent + source.total_spent) / (target.order_count + source.order_count)"
        })
        .whenNotMatchedInsert(values = {
            "customer_id": "source.customer_id",
            "payment_method": "source.payment_method",
            "order_count": "source.order_count",
            "total_spent": "source.total_spent",
            "avg_order_value": "source.avg_order_value"
        })
        .execute()
    )
    
    print("✓ Incremental MERGE wykonany!")

**MERGE incremental aggregates do summary table:**

Używamy MERGE aby zaktualizować istniejące aggregates lub wstawić nowe.

In [ ]:
if inserts_count > 0:
    incremental_agg = (inserts_only
        .groupBy("customer_id", "payment_method")
        .agg(
            F.count("*").alias("order_count"),
            F.sum("total_amount").alias("total_spent"),
            F.avg("total_amount").alias("avg_order_value")
        )
    )
    
    print(f"Incremental aggregates: {incremental_agg.count()} rows")
    display(incremental_agg)
else:
    print("Brak nowych insertów do przetworzenia")

**Oblicz incremental aggregates:**

Agregujemy tylko nowe rekordy (z CDF insert events) zamiast przetwarzać całą tabelę.

In [ ]:
inserts_only = changes_incremental.filter(F.col("_change_type") == "insert")
inserts_count = inserts_only.count()

print(f"Insert events to process: {inserts_count}")

**Filtruj tylko INSERT events (dla simplicity):**

W production należałoby obsłużyć wszystkie typy zmian (update, delete), ale dla demonstracji skupiamy się na insert.

In [ ]:
changes_incremental = (spark.read
    .format("delta")
    .option("readChangeFeed", "true")
    .option("startingVersion", last_processed_version + 1)
    .table(ORDERS_DELTA)
)

print(f"CDF changes (versions {last_processed_version + 1} to {current_version}):")
print(f"Change events: {changes_incremental.count()}")

**Odczytaj tylko changes od last processed version:**

CDF pozwala na efektywny odczyt tylko zmienionych rekordów między wersjami.

In [ ]:
current_version = spark.sql(f"DESCRIBE HISTORY {ORDERS_DELTA}") \
    .agg({"version": "max"}).collect()[0][0]

**Incremental processing z CDF:**

Zamiast przetwarzać całą tabelę, używamy CDF aby odczytać tylko zmiany od last processed version.

In [ ]:
new_orders_2.write.mode("append").saveAsTable(ORDERS_DELTA)
print("✓ Dodano 2 nowe zamówienia")

**Zapisz nowe zamówienia:**

Te zamówienia utworzą nowe events w CDF, które będziemy przetwarzać incrementally.

In [ ]:
new_orders_2 = spark.createDataFrame([
    ("ORD_CDF_003", "CUST001234", "PROD000789", "STORE030", "2025-01-23T09:00:00", 
     3, 66.66, 0, 199.98, "Credit Card"),
    ("ORD_CDF_004", "CUST001236", "PROD000111", "STORE040", "2025-01-23T10:00:00", 
     1, 299.99, 15, 254.99, "Cash")
], ["order_id", "customer_id", "product_id", "store_id", "order_datetime",
    "quantity", "unit_price", "discount_percent", "total_amount", "payment_method"])

**Symulacja nowych zmian w source table:**

Dodajemy kolejne zamówienia aby zademonstrować incremental processing z CDF.

In [ ]:
last_processed_version = spark.sql(f"DESCRIBE HISTORY {ORDERS_DELTA}") \
    .agg({"version": "max"}).collect()[0][0]
    
print(f"Last processed version: {last_processed_version}")

**Zapisz last processed version:**

Zapamiętujemy do której wersji przetworzyliśmy dane - będzie to baseline dla incremental processing.

In [ ]:
initial_summary.write.format("delta").mode("overwrite").saveAsTable(ORDERS_SUMMARY)

records_count = spark.table(ORDERS_SUMMARY).count()
print(f"✓ Utworzono {ORDERS_SUMMARY}")
print(f"Records: {records_count}")

**Zapisz summary table:**

Zapisujemy initial aggregation jako Delta table w Gold layer.

In [ ]:
initial_summary = (spark.table(ORDERS_DELTA)
    .groupBy("customer_id", "payment_method")
    .agg(
        F.count("*").alias("order_count"),
        F.sum("total_amount").alias("total_spent"),
        F.avg("total_amount").alias("avg_order_value")
    )
)

**Initial load - full aggregation:**

Tworzymy tabelę podsumowań z pełnej agregacji wszystkich obecnych zamówień.